In [1]:
import pandas as pd
import os

In [2]:
data = './data/kaggle/'
seed = 42

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import pandas as pd


data = './data/kaggle/'
seed = 42

## Prepare Dataset

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

train_data = pd.read_csv(data+'kaggle_train.csv')
test_data = pd.read_csv(data+'kaggle_test.csv')
test_ids = test_data.pop('ID')
train_ids = train_data.pop('ID')
train_y = train_data.pop('TARGET')
X_train, X_valid, y_train, y_valid = train_test_split(train_data, train_y, test_size=0.2, random_state=seed)

X = torch.Tensor(np.asarray(X_train))
Y = torch.Tensor(np.asarray(y_train)).unsqueeze(1)
training_loader = DataLoader(list(zip(X,Y)), shuffle=True, batch_size=bs)

X = torch.Tensor(np.asarray(X_valid))
Y = torch.Tensor(np.asarray(y_valid)).unsqueeze(1)
validation_loader = DataLoader(list(zip(X,Y)), shuffle=True, batch_size=bs)

print('Number of batches:', len(training_loader))

Number of batches: 205


## Define Network

In [33]:
lr = 0.001
momentum = 0.9
weight_decay = 0.01


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.input = nn.Linear(17, 60)
        self.act1 = nn.ReLU()
        self.hidden1 = nn.Linear(60, 60)
        self.act2 = nn.ReLU()
        self.hidden2 = nn.Linear(60, 60)
        self.act3 = nn.ReLU()
        self.output = nn.Linear(60,1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.input(x))
        x = self.act2(self.hidden1(x))
        x = self.act3(self.hidden2(x))
        x = self.sigmoid(self.output(x))
        return x
    
model = NeuralNetwork()
loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

## Training Loop
### Train for one epoch

In [34]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    for i, data in enumerate(training_loader):
        # Every data instance is an input / label pair
        inputs, labels = data
        # Zero the gradients at every batch
        optimizer.zero_grad()
        # Make predictions
        outputs = model(inputs)
        # Compute loss and gradient
        loss = loss_fn(outputs, labels)
        loss.backward()
        # Adjust learning weights
        optimizer.step()
        # Gather data and report
        running_loss += loss.item()
        if i % 10 == 9:
            last_loss = running_loss/10
            print(' batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(training_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0
    return last_loss

### Main training loop

In [35]:
from datetime import datetime
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('./runs/alcoholic_trainer_{}'.format(timestamp))
epoch_number = 0

EPOCHS = 50
best_vloss=1_000_000.

for epoch in range(EPOCHS):
    print('EPOCH {}'.format(epoch_number + 1))

    # Make sure gradient tracking is on and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(epoch_number, writer)

    running_vloss = 0.0
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    model.eval()

    # Disable gradient computation and reduce memory consumption
    with torch.no_grad():
        for i, vdata in enumerate(validation_loader):
            vinputs, vlabels = vdata
            voutputs = model(vinputs)
            vloss = loss_fn(voutputs, vlabels)
            running_vloss += vloss
    
    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Log the running loss averaged per batch
    # for both training and validation
    writer.add_scalars('Training vs. Validation Loss', { 'Training' : avg_loss, 'Validation' : avg_vloss }, epoch_number + 1)
    writer.flush()

    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = './bin/model_{}_{}'.format(timestamp, epoch_number)
        torch.save(model.state_dict(), model_path)
    
    epoch_number += 1

EPOCH 1
 batch 10 loss: 0.7029974818229675
 batch 20 loss: 0.6690834164619446
 batch 30 loss: 0.6972606539726257
 batch 40 loss: 0.6871707320213318
 batch 50 loss: 0.6919213950634002
 batch 60 loss: 0.6874413013458252
 batch 70 loss: 0.6917330622673035
 batch 80 loss: 0.7033853948116302
 batch 90 loss: 0.6895834982395173
 batch 100 loss: 0.6939397931098938
 batch 110 loss: 0.6829995155334473
 batch 120 loss: 0.6948353886604309
 batch 130 loss: 0.6981848835945129
 batch 140 loss: 0.6946260511875153
 batch 150 loss: 0.6896742582321167
 batch 160 loss: 0.6905553460121154
 batch 170 loss: 0.6811096727848053
 batch 180 loss: 0.6922442853450775
 batch 190 loss: 0.6981460988521576
 batch 200 loss: 0.6898232817649841
LOSS train 0.6898232817649841 valid 0.6935492157936096
EPOCH 2
 batch 10 loss: 0.6922598838806152
 batch 20 loss: 0.7023778975009918
 batch 30 loss: 0.6865395784378052
 batch 40 loss: 0.688350933790207
 batch 50 loss: 0.6913292288780213
 batch 60 loss: 0.6870447516441345
 batch 70

## Load best model and predict for test set

In [48]:
model = NeuralNetwork()
model.load_state_dict(torch.load('bin\model_20240222_130438_47'))

<All keys matched successfully>

In [59]:
def assign_score(row):
    row_tensor = torch.Tensor(row)
    with torch.no_grad():
        pred = round(float(model(row_tensor)))
    return pred


preds = test_data.apply(lambda row: assign_score(row), axis=1)
nn_submission = pd.DataFrame({'ID': test_ids, 'TARGET': preds})
nn_submission.to_csv('./results/WESLEY_MORRIS_neural_network.csv', index=False)

C:\Users\morriwg1\AppData\Local\Temp\ipykernel_19604\3267349410.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row_tensor = torch.Tensor(row)
